In [29]:
import pandas as pd
import re
from datasets import Dataset

In [59]:
data_path = "../data/BB_data.csv"
df = pd.read_csv(data_path)
# Remove any rows that contain NaN values
df = df.dropna()

In [60]:
df.head()

,actor,text,season,episode,title
0,Walter,My name is Walter Hartwell White. I live at 30...,1,1,The Pilot
1,Skyler,Happy Birthday.,1,1,The Pilot
2,Walter,Look at that.,1,1,The Pilot
3,Skyler,That is veggie bacon. Believe it or not. Zero ...,1,1,The Pilot
4,Walter,Same time.,1,1,The Pilot


In [61]:

df = df.drop(columns=["season"])
df = df.drop(columns=["episode"])
df = df.drop(columns=["title"])


# Save cleaned dataset (optional)
df.to_csv("bb_data_cleaned.csv", index=False)

print(df.head())

    actor                                               text
0  Walter  My name is Walter Hartwell White. I live at 30...
1  Skyler                                    Happy Birthday.
2  Walter                                      Look at that.
3  Skyler  That is veggie bacon. Believe it or not. Zero ...
4  Walter                                         Same time.


In [62]:
df['number_of_words'] = df['text'].str.strip().str.split(" ")
df['number_of_words'] = df['number_of_words'].apply(lambda x: len(x) if isinstance(x, list) else 0)

In [63]:
df.head()

,actor,text,number_of_words
0,Walter,My name is Walter Hartwell White. I live at 30...,96
1,Skyler,Happy Birthday.,2
2,Walter,Look at that.,3
3,Skyler,That is veggie bacon. Believe it or not. Zero ...,24
4,Walter,Same time.,2


In [64]:
df['walter_response_flag'] = 0
df.loc[(df['actor']=="Walter")&(df['number_of_words']>5),'walter_response_flag']=1

In [65]:
indexes_to_take = df.index[df['walter_response_flag'] == 1].tolist()

In [66]:
indexes_to_take[:3]

[0, 14, 16]

In [67]:
system_promt = """" Your are Walter White from the TV series "Breaking Bad". Your responses should reflect his personality and speech patterns \n"""

prompts = []
for ind in indexes_to_take:
    prompt = system_promt

    prompt += df.iloc[ind -1]['text']
    prompt += '\n'
    prompt += df.iloc[ind]['text']
    prompts.append(prompt)

In [71]:
print(prompts[1])


" Your are Walter White from the TV series "Breaking Bad". Your responses should reflect his personality and speech patterns 
Did you take your Echinacea?
Yeah. I think it's getting better.


In [72]:
df = pd.DataFrame({"prompt":prompts})
df.head()

,prompt
0,""" Your are Walter White from the TV series ""Br..."
1,""" Your are Walter White from the TV series ""Br..."
2,""" Your are Walter White from the TV series ""Br..."
3,""" Your are Walter White from the TV series ""Br..."
4,""" Your are Walter White from the TV series ""Br..."


In [74]:
dataset = Dataset.from_pandas(df)